# Import Statement 

In [1]:
import os
import import_ipynb
from emotion_inference import predict_emotion as _model_predict_emotion
from dialogue_inference import predict_act as _model_predict_act
from typing import List, Dict, Tuple
from openai import OpenAI

<string>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
<string>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which 

In [2]:
# Set your api key here.
os.environ["OPENAI_API_KEY"] = "Enter your Api key here"
os.environ.setdefault("OPENAI_MODEL", "gpt-4o-mini")

# Initializing client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Selecting the model
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")


# Defining tone mapping

In [3]:
EMOTION_TONE = {
    "anger":   "calm, validating, non-confrontational",
    "sadness": "warm, reassuring, supportive",
    "disgust":    "patient, reflective, balanced",
    "happiness":     "positive, concise, appreciative",
    "fear":    "gentle, grounding, confidence-building",
    "surprise":"curious, clarifying, steady",
    "neutral": "matter-of-fact, clear",
}

DIALOGUE_ACT_STYLE = {
    "question": "answer clearly, then ask one helpful follow-up if useful",
    "inform": "be concise and structured. answer in one sentence",
    "directive": "acknowledge and follow whats specified.",
    "commisive": "confirm next steps, express encouragement, and offer assistance if needed.",
}


# Plugging in pre-trained models

In [4]:
# Predicting emotion from input text using pre-trained emotion classification model.
def predict_emotion_from_model(text, history=None):
    res = _model_predict_emotion(text)
    label = (res.get("pred") or "").strip().lower()
    prob  = float(res.get("prob", 0.0))
    if label == "no emotion": label = "neutral"
    if label not in {"neutral","anger","disgust","fear","happiness","sadness","surprise"}:
        label = "neutral"
    return label, prob

# Predicting dialogue act from input text using pre-trained dialogue act classification model.
def predict_act_from_model(text, history=None):
    res = _model_predict_act(text)
    label = (res.get("pred") or "INFORM").upper()
    prob = float(res.get("prob", 0.0))
    # Clamp to the set you use in the engine
    if label not in {"QUERY","DIRECT","COMMISSIVE","INFORM"}:
        label = "INFORM"
    return label, prob

# Prompt Engineering

In [5]:
def _select_history(history, max_turns=3):
    return history[-max_turns:]

def _bullet_history(history):
    lines = []
    for t in reversed(history):
        who = "User" if t["role"] == "user" else "Assistant"
        lines.append(f"- {who}: {t['text']}")
    return "\n".join(lines)

# Constructing message for gpt-4o-mini model.
def build_messages(user_utt, history, emotion, e_prob, act, a_prob, conditioning=None):
    cond = conditioning or {"use_emotion": True, "use_act": True}

    # Fetching tone from detected emotion
    tone = (
        EMOTION_TONE.get(emotion, "clear and respectful")
        if cond.get("use_emotion", True)
        else "clear and respectful"
    )

    # Fetching style from detected dialogue act
    act_style = (
        DIALOGUE_ACT_STYLE.get(act, "neutral and professional")
        if cond.get("use_act", True)
        else "neutral and professional"
    )

    sys_msg = {
        "role": "system",
        "content": (
            "You are a concise, empathetic assistant.\n"
            "Goals: (1) Be correct and helpful."
            "(2) STRICTLY match the provided emotional tone when available. "
            "(3) STRICTLY follow the provided dialogue-act style when available.\n"
        )
    }

    body = [
        "[CONTEXT]",
        f'Latest user utterance: "{user_utt}"',
        "Dialogue history (most recent first):",
        _bullet_history(_select_history(history)),
        ""
    ]

    # Adding emotion condition to prompt.
    if cond.get("use_emotion", True):
        body.append(f"Predicted emotion: {emotion} (p={e_prob:.2f})")
        body.append(f"TONE TO USE (STRICT): {tone}")
    else:
        body.append("Emotion conditioning: DISABLED")
        body.append("TONE TO USE (STRICT): clear and respectful")

    # Adding act condition to prompt.
    if cond.get("use_act", True):
        body.append(f"Predicted dialogue act: {act} (p={a_prob:.2f})")
        body.append(f"STYLE TO USE (STRICT): {act_style}")
    else:
        body.append("Dialogue-act conditioning: DISABLED")
        body.append("STYLE TO USE (STRICT): neutral and professional")

    body.append(
        "Combine tone + style: tone controls affect/wording; style controls structure/response pattern. "
        "Do not override either when provided."
    )
    body.append("Never fabricate facts. If uncertain, ask one crisp clarifying question.")

    dev_msg = {"role": "developer", "content": "\n".join(body)}
    user_msg = {"role": "user", "content": user_utt}
    return [sys_msg, dev_msg, user_msg]


# API call to gpt-4o-mini model

In [6]:
# Making request to gpt-4o-mini model.
def llm_chat(messages, model=MODEL, temperature=0.6, max_tokens=300):
    resp = client.chat.completions.create(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=messages
    )
    return resp.choices[0].message.content

# Response Engine

In [7]:
# Class to manages the complete conversation flow.
class ResponseEngine:
    def __init__(self, predict_emotion_fn, predict_act_fn, llm_chat_fn, max_history=6, conditioning=None):
        self.predict_emotion_fn = predict_emotion_fn
        self.predict_act_fn = predict_act_fn
        self.llm_chat_fn = llm_chat_fn
        self.history = []
        self.max_history = max_history
        self.cond = conditioning or {"use_emotion": True, "use_act": True}

    def reset(self):
        self.history = []

    def step(self, user_text, return_debug=False):
        self.history.append({"role": "user", "text": user_text})
        self.history = self.history[-self.max_history:]

        emotion, e_prob = self.predict_emotion_fn(user_text, self.history)
        act, a_prob = self.predict_act_fn(user_text, self.history)

        messages = build_messages(user_text, self.history, emotion, e_prob, act, a_prob, self.cond)
        assistant_text = self.llm_chat_fn(messages).strip()

        self.history.append({"role": "assistant", "text": assistant_text})
        self.history = self.history[-self.max_history:]

        if return_debug:
            return {
                "user": user_text,    
                "reply": assistant_text,
                "emotion": (emotion, e_prob),
                "act": (act, a_prob),
                "conditioning": dict(self.cond)
            }
        return assistant_text


In [8]:
# Function to produce sample outputs.
def sample_demo(engine):
    for utterance in [
        "How’s the weather today? Hopefully it’s not terrible again.",
        "Please remind me to call Monica at 5 PM so I can share the good news.",
        "Where is the nearest coffee shop?",
        "I live near Central Perk in New York."
        
    ]:
        out = engine.step(utterance, return_debug=True)
        print(f"User: {out['user']}")
        print(f"Assistant: {out['reply']}")
        print(f"(emotion={out['emotion'][0]} p={out['emotion'][1]:.2f} | act={out['act'][0]} p={out['act'][1]:.2f} | conditioning={out['conditioning']})\n")


# Ablation Studies

## Response Generation with  both emotion and dialogue act conditioning.

In [9]:
print("Response Generation with  both emotion and dialogue act conditioning.")
print()
engine = ResponseEngine(
predict_emotion_fn=predict_emotion_from_model,
predict_act_fn=predict_act_from_model,
llm_chat_fn=llm_chat,
conditioning={"use_emotion": True, "use_act": True}
)
sample_demo(engine)
print("*"*150)

   

Response Generation with  both emotion and dialogue act conditioning.

User: How’s the weather today? Hopefully it’s not terrible again.
Assistant: Today’s weather is mild with some clouds, so it should be an improvement from recent days.
(emotion=anger p=0.85 | act=inform p=0.65 | conditioning={'use_emotion': True, 'use_act': True})

User: Please remind me to call Monica at 5 PM so I can share the good news.
Assistant: Got it! I'll remind you to call Monica at 5 PM to share your good news. That's exciting!
(emotion=happiness p=0.75 | act=directive p=0.74 | conditioning={'use_emotion': True, 'use_act': True})

User: Where is the nearest coffee shop?
Assistant: To help you find the nearest coffee shop, could you please provide your current location or the name of a nearby area?
(emotion=neutral p=0.60 | act=question p=0.72 | conditioning={'use_emotion': True, 'use_act': True})

User: I live near Central Perk in New York.
Assistant: The nearest coffee shop to Central Perk is likely the S

## Response Generation with emotion conditioning only

In [10]:
print("Response Generation with emotion conditioning only")
print()
engine = ResponseEngine(
predict_emotion_fn=predict_emotion_from_model,
predict_act_fn=predict_act_from_model,
llm_chat_fn=llm_chat,
conditioning={"use_emotion": True, "use_act": False}
)
sample_demo(engine)
print("*"*150)

Response Generation with emotion conditioning only

User: How’s the weather today? Hopefully it’s not terrible again.
Assistant: I understand your frustration with the weather. Today, it looks like there will be a mix of clouds and sun, with mild temperatures. Hopefully, it will be more pleasant for you.
(emotion=anger p=0.85 | act=inform p=0.65 | conditioning={'use_emotion': True, 'use_act': False})

User: Please remind me to call Monica at 5 PM so I can share the good news.
Assistant: I can't set reminders, but you could use your phone or a reminder app to help with that! Sharing good news is always exciting—I'm sure Monica will be thrilled!
(emotion=happiness p=0.75 | act=directive p=0.74 | conditioning={'use_emotion': True, 'use_act': False})

User: Where is the nearest coffee shop?
Assistant: To find the nearest coffee shop, you can use a map application on your phone or computer. Just enter "coffee shop" in the search bar, and it will show you the closest options along with direc

## Response Generation with dialogue act conditioning only

In [11]:
print("Response Generation with dialogue act conditioning only")
print()
engine = ResponseEngine(
predict_emotion_fn=predict_emotion_from_model,
predict_act_fn=predict_act_from_model,
llm_chat_fn=llm_chat,
conditioning={"use_emotion": False, "use_act": True}
)
sample_demo(engine)
print("*"*150)

Response Generation with dialogue act conditioning only

User: How’s the weather today? Hopefully it’s not terrible again.
Assistant: I cannot provide real-time weather updates, but you can check a reliable weather website or app for the latest conditions in your area.
(emotion=anger p=0.85 | act=inform p=0.65 | conditioning={'use_emotion': False, 'use_act': True})

User: Please remind me to call Monica at 5 PM so I can share the good news.
Assistant: I can't set reminders, but you might want to set an alarm on your phone or use a reminder app for that.
(emotion=happiness p=0.75 | act=directive p=0.74 | conditioning={'use_emotion': False, 'use_act': True})

User: Where is the nearest coffee shop?
Assistant: I don’t have access to your location, but you can easily find the nearest coffee shop using a maps app on your phone or by searching online. Would you like tips on how to use those apps?
(emotion=neutral p=0.60 | act=question p=0.72 | conditioning={'use_emotion': False, 'use_act': T

## Response Generation without Conditioning

In [12]:
print("Response Generation without Conditioning")
print()
engine = ResponseEngine(
predict_emotion_fn=predict_emotion_from_model,
predict_act_fn=predict_act_from_model,
llm_chat_fn=llm_chat,
conditioning={"use_emotion": False, "use_act": False}
)
sample_demo(engine)
print("*"*150)

Response Generation without Conditioning

User: How’s the weather today? Hopefully it’s not terrible again.
Assistant: I currently don't have real-time weather information. I recommend checking a reliable weather website or app for the most accurate update. If you’d like, I can help you with general tips on how to prepare for different weather conditions.
(emotion=anger p=0.85 | act=inform p=0.65 | conditioning={'use_emotion': False, 'use_act': False})

User: Please remind me to call Monica at 5 PM so I can share the good news.
Assistant: I can't set reminders, but you might consider using a reminder app or setting an alarm on your phone for 5 PM to call Monica.
(emotion=happiness p=0.75 | act=directive p=0.74 | conditioning={'use_emotion': False, 'use_act': False})

User: Where is the nearest coffee shop?
Assistant: I don't have access to real-time location data. However, you can use a map application on your phone or computer to find the nearest coffee shop. Would you like tips on ho